In [34]:
# load environment variables
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

# selenium to parse dynamic web pages
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# import utilities
import pandas as pd
import time

# Authentication & Filters

In [35]:
# get environment variables
card_num = os.environ.get('MISS_LIB_CARD')
base_url = os.getenv('BASE_URL')

# search terms
query = "banquet"
data_values = [
    581212, # Banquet Facilities
    791102, # Ballrooms
    799904, # Auditoriums
    738931] # Convention & Meeting Facilities

In [36]:
print(base_url)

https://www.mississauga.ca/library/research-and-learn/business/


# Helper Functions

In [37]:
def document_initialised(driver):
    '''
    document_initialised will command the driver to wait for the page to load completely.

    Args:
        driver (ChromeService object): Chrome browser driver.

    Returns:
        js script: Driver executes a script to wait for the page to load or, if it has already loaded before the listener can catch it, to proceed.
    '''

    return driver.execute_script("if (document.readyState === 'complete') {return true;} else {window.addEventListener('load', () => {return true;});}")

def wait_for_page(n):
    '''
    wait_for_page will stall the selenium driver to listen for conditions.

    Args:
        n (int): Set wait time in seconds.

    '''

    WebDriverWait(driver, timeout=10).until(document_initialised)
    time.sleep(n)
    return



# Scraper Functions

In [38]:
def scrape_table_data(driver):
    '''
    scrape_page_data will scrape all data from the tabulated, high-level results.
    NOTE: This function only works if it is called on the first page of the tabulated results.

    Args:
        driver (WebDriver): Chrome browser driver.

    Returns:
        data (list): Scraped entries from the tables.
        heading_names (list): Scraped fields from the tables.
    '''

    # find the resulting number of pages
    max_pages = driver.find_element(By.CLASS_NAME, "data-page-max").text

    # define containers
    data = list()
    heading_names = list()

    # scrape table headings
    headings = driver.find_elements(By.XPATH, "//thead[@id = 'searchResultsHeader']/tr/th")
    for heading in headings:
        heading_names.append(heading.text)

    for _ in range(1, int(max_pages)+1):
        # scrape entries from table
        rows = driver.find_elements(By.XPATH, "//tbody[@id = 'searchResultsPage']/tr")

        for row in rows:
            row_data = list()
            for col in row.find_elements(By.TAG_NAME, "td"):
                row_data.append(col.text)
            data.append(row_data)

        # go to the next page then wait
        driver.find_elements(By.CLASS_NAME, "next")[0].click()
        wait_for_page(3)

    return data, heading_names

def scrape_groupbox_info(groupbox, groupbox_title):
    '''
    scrape_groupbox_info will scrape groupbox details from one company page.

    Args:
        groupbox (WebDriver): WebDriver element filtering for groupbox class divs.
        groupbox_title (WebDriver element): WebDriver element filtering for groupbox_title divs.

    Returns:
        temp_list (list): List of data values scraped from each groupbox.
        count (int): Length of items in temp_list.
    '''

    temp_list = list()

    # find all information in the groupbox
    cols = groupbox.find_elements(By.TAG_NAME, "td")
    count = len(cols)

    # append information to the info_list
    for col in cols:
        temp_list.append(col.text)

    # capture if website if present, return nothing if not
    # try:
    #     temp_list.append(groupbox.find_element(By.TAG_NAME, "a").get_attribute("href"))
    # except NoSuchElementException:
    #     temp_list.append("")

    return temp_list, count

def scrape_company_page(groupboxes, groupbox_titles):

    info_list = list()

    # Isolate site for each group box based on title
    for i, groupbox_title in enumerate(groupbox_titles):
        # Switch case for each groupbox based on their titles
        match groupbox_title.text:
            case "Location Information":
                temp_list, count = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)
                # check if address is two lines and add blank item if not
                if temp_list[3] == "":
                    info_list.insert(2, "")
                    count += 1 # adds another line for the benefit of the website availability check below
                # check if website is included in the information and add blank if not
                if count < 10:
                    info_list.append("")

            case "Industry Profile":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.append(temp_list)

            case "Photo, Map, & Directions":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)

            case "Business Demographics":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)

            case "Management Directory":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.append(temp_list)

            case "Business Expenditures":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)

            case "Nearby Businesses":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)

            case "Competitors Report":
                temp_list, _ = scrape_groupbox_info(groupboxes[i], groupbox_title)
                info_list.extend(temp_list)

            case _:
                print("")

    return info_list

# Set Browser Options & Install Drivers

In [39]:
# define options for browser
options = ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')

# install driver and open URL
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# Navigate to URL to Results Table

In [40]:
# navigate to URL
driver.get(base_url)

# navigate to Canadian Businesses
wait_for_page(1)
driver.find_element(By.XPATH, ("//a[@title = 'Canadian Businesses']")).click()

# check off on the ToC and Agree
wait_for_page(1)
driver.find_element(By.ID, "chkAgree").click()
driver.find_element(By.CLASS_NAME, "action-agree").click()

# enter card number and log on
wait_for_page(1)
driver.find_element(By.ID, "matchcode").send_keys(card_num)
driver.find_element(By.CLASS_NAME, "action-submit-form").click()

# open Canadian Business search
wait_for_page(1)
driver.find_element(By.CLASS_NAME, ("action-do-search")).click()

# switch to Advanced Search and interact with filters
wait_for_page(1)
driver.find_element(By.CLASS_NAME, "advancedSearch").click()
driver.find_element(By.ID, "cs-YellowPageHeadingOrSic").click() # Business Type: Keyword/SIC/NAICS
driver.find_element(By.ID, "cs-Province").click() # Geography: Province

# wait for advanced filter options to populate
wait_for_page(1)
driver.find_element(By.CSS_SELECTOR, "input#VerifiedOnly").click()
driver.find_element(By.CSS_SELECTOR, "input#sicPrimaryOptionId").click() # radio: Search primary SIC Only
driver.find_element(By.ID, "sicLookupKeyword").send_keys(query) # search field: Query
driver.find_element(By.ID, "searchSic").click() # search button: Query

# wait for results to populate and highlight chosen query fields
wait_for_page(3)
for value in data_values:
    driver.find_element(By.XPATH, f"//li[@data-value = {value}]").click() # results: Highlight NAICS results
driver.find_element(By.XPATH, f"//li[@data-value = 'ON']").click() # results: Highlight Ontario results
driver.find_element(By.CLASS_NAME, "action-view-results").click() # View Results with filters

# results page
wait_for_page(3)
driver.find_element(By.XPATH, "//*[@alt = 'Optional Column']").click()
driver.find_element(By.XPATH, "//*[@data-key = 'Title']").click()

In [41]:
data = list()
# find the resulting number of pages
max_pages = int(driver.find_element(By.CLASS_NAME, "data-page-max").text)

# iterate through pages
for i in range(max_pages):

    info_list = list()

    # find the resulting number of companies per page
    company_links = driver.find_elements(By.XPATH, "//tbody[@id ='searchResultsPage']/tr/td/a")

    # iterate through each company page and scrape each page by groupbox
    for j, _ in enumerate(company_links):
        print(f"Gathering info on page {i+1}, row {j+1}...")
        # enter a company page
        wait_for_page(3)
        company = driver.find_elements(By.XPATH, "//tbody[@id ='searchResultsPage']/tr/td/a")
        try:
            company[j].click()
        except IndexError:
            print(f"List item {i+1} on page {j+1} cannot be found")
        # scrape company page and append to existing list
        wait_for_page(3)
        groupboxes = driver.find_elements(By.CLASS_NAME, "groupbox")
        groupbox_titles = driver.find_elements(By.CLASS_NAME, "groupboxTitle")
        info_list.append(scrape_company_page(groupboxes, groupbox_titles))
        # navigate back to results table page
        driver.back()

    wait_for_page(3)
    data.extend(info_list)
    try:
        driver.find_elements(By.CLASS_NAME, "next")[0].click()
    except:
        print("End of results.")

Gathering info on page 1, row 1...
Gathering info on page 1, row 2...
Gathering info on page 1, row 3...
Gathering info on page 1, row 4...
Gathering info on page 1, row 5...
Gathering info on page 1, row 6...
Gathering info on page 1, row 7...
Gathering info on page 1, row 8...
Gathering info on page 1, row 9...
Gathering info on page 1, row 10...
Gathering info on page 1, row 11...
Gathering info on page 1, row 12...
Gathering info on page 1, row 13...
Gathering info on page 1, row 14...
Gathering info on page 1, row 15...
Gathering info on page 1, row 16...
Gathering info on page 1, row 17...
Gathering info on page 1, row 18...
Gathering info on page 1, row 19...
Gathering info on page 1, row 20...
Gathering info on page 1, row 21...
Gathering info on page 1, row 22...
Gathering info on page 1, row 23...
Gathering info on page 1, row 24...
Gathering info on page 1, row 25...
Gathering info on page 2, row 1...
Gathering info on page 2, row 2...
Gathering info on page 2, row 3...
Gath

IndexError: list index out of range

In [43]:
print(len(data))
data

1125


[['10 Tation Event Catering',
  '',
  '232 Norseman St',
  'Etobicoke, ON M8Z2R4',
  '',
  '(416) 243-5144',
  'Toronto Metropolitan',
  '(416) 243-2662',
  'Toronto',
  '10tation.com',
  ['5812-12',
   'Caterers',
   '',
   'Regular',
   '2019',
   '72232001',
   'Caterers',
   '',
   'Regular',
   '2019'],
  '24',
  '$1,296,000',
  'Not Available',
  'Not Available',
  'Single Loc',
  'Excellent',
  '',
  '43.632250 / -79.529510',
  '03-839-5380',
  'No',
  ["David D'Aprile", 'Owner', 'Male'],
  'These expenditures are an estimated annual expense.',
  '$2,500 to $5,000',
  '$20,000 to $50,000',
  '$1,000 to $10,000',
  '$10,000 to $25,000',
  '$1,000 to $2,500',
  '$5,000 to $10,000',
  '$20,000 to $50,000',
  '$10,000 to $25,000',
  '$250,000 to $500,000',
  '$1,000 to $2,500',
  '$50,000 to $100,000',
  '$1,000 to $2,500',
  '$2,000 to $5,000',
  '$25,000 to $50,000'],
 ['1595645 Ontario Inc',
  '',
  '2155a Steeles Ave E',
  'Brampton, ON L6T5A1',
  '',
  '(905) 450-9996',
  'Peel

In [44]:
for i, line in enumerate(data):
    if len(line) == 0:
        print(i, 0, line)
    if len(line) == 38:
        print(i, 38, line)
    if len(line) == 39:
        print(i, 39, line)

1110 0 []
1111 0 []
1112 0 []
1113 0 []
1114 0 []
1115 0 []
1116 0 []
1117 0 []
1118 0 []
1119 0 []
1120 0 []
1121 0 []
1122 0 []
1123 0 []
1124 0 []


In [46]:
headings = [
    'Company Name',
    '',
    'Address Line 1',
    'Address Line 2',
    '',
    'Phone',
    'District',
    'Fax',
    'CMA',
    'Website',
    'Industry Profile',
    'Location Employees',
    'Location Sales Volume',
    'Corporate Employees',
    'Corporate Sales Volume',
    'Location Type',
    'Credit Rating',
    'Stock Ticker Table',
    'Latitude/Longitude',
    'IUSA Number',
    'Federal Contractor',
    'Management Directory',
    'Disclosure',
    'Accounting',
    'Advertising',
    'Contract Labor',
    'Insurance',
    'Legal',
    'Management/Administration',
    'Office Equipment & Supplies',
    'Package/Container',
    'Payroll & Benefits',
    'Purchased Print',
    'Rent & Leasing',
    'Technology',
    'Telecommunications',
    'Utilities'
]

In [47]:
df = pd.DataFrame(data, columns=headings)
df.to_excel('lead-list-detailed.xlsx')

In [49]:
df.tail(50)

,Company Name,,Address Line 1,Address Line 2,,Phone,District,Fax,CMA,Website,...,Legal,Management/Administration,Office Equipment & Supplies,Package/Container,Payroll & Benefits,Purchased Print,Rent & Leasing,Technology,Telecommunications,Utilities
1075,Vimika Banquet & Catering,,80 Nashdene Rd,"Scarborough, ON M1V5E4",,(416) 704-9755,Toronto Metropolitan,Not Available,Toronto,,...,Less than $500,"Less than $2,500","$5,000 to $10,000","$1,000 to $5,000","Less than $100,000",Less than $500,"$10,000 to $25,000",Less than $500,"Less than $2,000","$5,000 to $10,000"
1076,Vimika Banquets,,21-2901 Markham Rd,"Scarborough, ON M1X0B6",,(416) 428-3333,Toronto Metropolitan,Not Available,Toronto,,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
1077,Vinci Catering-Event Planning,,168 The Queensway,"Etobicoke, ON M8Y1J3",,(416) 251-4141,Toronto Metropolitan,Not Available,Toronto,vincicatering.ca,...,Less than $500,"Less than $2,500","Less than $5,000","$1,000 to $5,000","Less than $100,000",Less than $500,"Less than $10,000",Less than $500,"Less than $2,000","$2,000 to $5,000"
1078,VIP Catering & Janitorial,,801-199 Larch St,"Sudbury, ON P3E5R1",,(705) 671-2395,Sudbury Regional,Not Available,Greater Sudbury,vipservices.ca,...,Less than $500,"Less than $2,500","$5,000 to $10,000","$1,000 to $5,000","Less than $100,000",Less than $500,"$10,000 to $25,000",Less than $500,"Less than $2,000","$5,000 to $10,000"
1079,Wanderers Rugby Club,,Harwood N,"Pickering, ON L1VL1V",,(905) 686-0957,Durham Regional,Not Available,Toronto,ajaxwanderers.com,...,Less than $500,"Less than $2,500","Less than $5,000","$1,000 to $5,000","Less than $100,000",Less than $500,"$10,000 to $25,000",Less than $500,"Less than $2,000","$5,000 to $10,000"
1080,Water St Cooker,,2084 Old Lakeshore Rd,"Burlington, ON L7R1A3",,(905) 634-2084,Halton Regional,Not Available,Hamilton,waterstreetcooker.ca,...,"$1,000 to $2,500","$5,000 to $10,000","$10,000 to $20,000","$5,000 to $10,000","$100,000 to $250,000","$500 to $1,000","$25,000 to $50,000","$1,000 to $2,500","$2,000 to $5,000","$10,000 to $25,000"
1081,Waterloo Region Police Assn,,1128 Rife Rd,"Cambridge, ON N1R5S3",,(519) 622-0771,Waterloo Regional,(519) 622-5194,Kitchener-Cambridge-Wtrlo,wrpa.org,...,Not Available,"$5,000 to $10,000","$5,000 to $10,000","$1,000 to $5,000","$100,000 to $250,000",Not Available,"$25,000 to $50,000","$500 to $1,000","Less than $2,000","$10,000 to $25,000"
1082,Webb's Catering,,4161 89 Hwy,"Cookstown, ON L0L1L0",,(705) 458-1563,Simcoe County,(705) 458-1209,Barrie,,...,Less than $500,"Less than $2,500","Less than $5,000","$500 to $1,000","Less than $100,000",Less than $500,"Less than $10,000",Less than $500,"Less than $2,000","$2,000 to $5,000"
1083,Weddings By Lars,,398 Division St,"Kingston, ON K7K4A7",,(613) 484-1842,Frontenac County,Not Available,Kingston,,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
1084,Welcome To The City,,100 Dupont St,"Toronto, ON M5R1V2",,(416) 924-4778,Toronto Metropolitan,Not Available,Toronto,,...,"$1,000 to $2,500","$5,000 to $10,000","$10,000 to $20,000",Less than $500,"$100,000 to $250,000","$10,000 to $25,000","Less than $10,000","$2,500 to $5,000","$2,000 to $5,000","Less than $2,000"
